In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import time
from tqdm import tqdm

from scipy import stats
from sklearn.model_selection import KFold

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn import linear_model


np.random.seed(42)

In [2]:
version='v2'

# Data Load

In [3]:
train = pd.read_excel("../data/train_clst.xlsx")
train = train.drop(['Unnamed: 0'], axis=1)
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,clst
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,3
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0,3
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0,3
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0,3
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1,1,2
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1,1,2
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1,1,2
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1,1,2


In [4]:
test = pd.read_excel("../data/test_clst.xlsx")
test = test.drop(['Unnamed: 0'], axis=1)
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,clst
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN,3
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN,3
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0,2
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [5]:
#train = pd.read_csv("../data/train.csv", encoding='cp949')

In [6]:
#train.describe()

In [7]:
#train.info()

In [8]:
#test = pd.read_csv("./data/test.csv", encoding='cp949')

In [9]:
#test.describe()

In [10]:
#test.info()

In [11]:
submission = pd.read_csv('../submission/sample_submission.csv', encoding='cp949')

In [12]:
submission_cat = submission.copy()
submission_lgbm = submission.copy()
submission_xgb = submission.copy()

# Feature Engineering

In [13]:
#시간 변수와 요일 변수를 추가해봅니다.
def time(x):
    return int(x[-2:])
train['hour'] = train['date_time'].apply(lambda x: time(x))
test['hour'] = test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday'] = train['date_time'].apply(lambda x :weekday(x))
test['weekday'] = test['date_time'].apply(lambda x :weekday(x))

train['weekend'] = train['weekday'].apply(lambda x : 1 if (x==5 or x==6) else 0)
test['weekend'] = test['weekday'].apply(lambda x : 1 if (x==5 or x==6) else 0)

def month(x):
    return int(x[5:7])
train['month'] = train['date_time'].apply(lambda x :month(x))
test['month'] = test['date_time'].apply(lambda x :month(x))

In [14]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice = {}
hot = {}
count = 0
for i in tqdm(range(0, len(train), len(train)//60)):
    count += 1
    ice[count] = train.loc[i,'비전기냉방설비운영']
    hot[count] = train.loc[i,'태양광보유']

100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 7496.30it/s]


In [15]:
for i in tqdm(range(len(test))):
    test.loc[i, '비전기냉방설비운영'] = ice[test['num'][i]]
    test.loc[i, '태양광보유'] = hot[test['num'][i]]

100%|██████████████████████████████████████████████████████████████████████████| 10080/10080 [00:09<00:00, 1051.55it/s]


In [16]:
test = test.interpolate(method='values')

In [17]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,clst,hour,weekday,weekend,month
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0,3,0,1,0,8
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0,3,1,1,0,8
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0,3,2,1,0,8
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0,3,3,1,0,8
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0,3,4,1,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0,2,19,0,0,8
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0,2,20,0,0,8
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,2,21,0,0,8
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,2,22,0,0,8


In [18]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

train_cdhs = np.array([])
for num in tqdm(range(1, 61)):
    temp = train[train['num'] == num]
    cdh = CDH(temp['기온(°C)'].values)
    train_cdhs = np.concatenate([train_cdhs, cdh])
train['CDH'] = train_cdhs

test_cdhs = np.array([])
for num in tqdm(range(1, 61)):
    temp = test[test['num'] == num]
    cdh = CDH(temp['기온(°C)'].values)
    test_cdhs = np.concatenate([test_cdhs, cdh])
test['CDH'] = test_cdhs

100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 516.94it/s]


In [19]:
train['THI'] = 9/5*train['기온(°C)'] - 0.55*(1-train['습도(%)']/100)*(9/5*train['기온(°C)']-26)+32
train['THI_cat'] = pd.cut(train['THI'], bins = [0, 68, 75, 80, 200], labels = [1,2,3,4])

test['THI'] = 9/5*test['기온(°C)'] - 0.55*(1-test['습도(%)']/100)*(9/5*test['기온(°C)']-26)+32
test['THI_cat'] = pd.cut(test['THI'], bins = [0, 68, 75, 80, 200], labels = [1,2,3,4])

In [20]:
train['태양광'] =  train['일조(hr)'] * train['태양광보유'] * train['강수량(mm)'].map(lambda x : np.exp(-x)) * train['풍속(m/s)'] / train['습도(%)'] * 100
test['태양광'] =  test['일조(hr, 3시간)'] * test['태양광보유'] * test['강수량(mm, 6시간)'].map(lambda x : np.exp(-x)) * test['풍속(m/s)'] / test['습도(%)'] * 100

In [21]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,clst,hour,weekday,weekend,month,CDH,THI,THI_cat,태양광
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,3,0,0,0,6,-8.4,63.43008,1,0.000000
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0,3,1,0,0,6,-16.7,63.56993,1,0.000000
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0,3,2,0,0,6,-25.2,63.22775,1,0.000000
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0,3,3,0,0,6,-34.1,62.54339,1,0.000000
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0,3,4,0,0,6,-43.1,62.39760,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1,1,2,19,0,0,8,30.8,77.80896,3,2.367647
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1,1,2,20,0,0,8,32.3,77.44917,3,0.000000
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1,1,2,21,0,0,8,32.5,77.44917,3,0.000000
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1,1,2,22,0,0,8,31.3,77.52246,3,0.000000


In [22]:
train.describe()

,num,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,clst,hour,weekday,weekend,month,CDH,THI,태양광
count,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.00000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000
mean,30.500000,2324.830866,24.251713,2.151641,80.169848,0.514989,0.213533,0.683333,0.483333,1.95000,11.500000,2.964706,0.282353,6.929412,-21.071605,73.461061,0.418172
std,17.318173,2058.999326,3.407902,1.514475,15.525862,2.624505,0.370517,0.465178,0.499724,0.82513,6.922215,2.014351,0.450146,0.793925,34.111471,4.778330,1.377256
min,1.000000,0.000000,11.100000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,6.000000,-151.300000,52.344210,0.000000
25%,15.750000,1055.268000,21.800000,1.100000,70.000000,0.000000,0.000000,0.000000,0.000000,1.00000,5.750000,1.000000,0.000000,6.000000,-45.300000,70.035960,0.000000
50%,30.500000,1700.352000,24.200000,1.900000,84.000000,0.000000,0.000000,1.000000,0.000000,2.00000,11.500000,3.000000,0.000000,7.000000,-20.800000,73.615850,0.000000
75%,45.250000,2780.487000,26.500000,2.900000,93.000000,0.000000,0.300000,1.000000,1.000000,3.00000,17.250000,5.000000,1.000000,8.000000,2.800000,77.074510,0.000000
max,60.000000,17739.225000,36.300000,20.100000,100.000000,81.500000,1.000000,1.000000,1.000000,3.00000,23.000000,6.000000,1.000000,8.000000,99.200000,86.661790,20.000000


In [23]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,clst,hour,weekday,weekend,month,CDH,THI,THI_cat,태양광
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0,3,0,1,0,8,1.800000,78.602280,3,0.000000
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0,3,1,1,0,8,3.433333,78.519273,3,0.000000
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0,3,2,1,0,8,4.900000,78.431867,3,0.000000
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0,3,3,1,0,8,6.200000,78.340060,3,0.000000
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0,3,4,1,0,8,7.100000,77.912697,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0,2,19,0,0,8,22.400000,78.764020,3,2.882155
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0,2,20,0,0,8,25.400000,78.364280,3,1.525705
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,2,21,0,0,8,27.700000,77.957280,3,0.000000
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,2,22,0,0,8,29.300000,77.957280,3,0.000000


In [24]:
test.describe()

,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,clst,hour,weekday,weekend,month,CDH,THI,태양광
count,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.00000,10080.000000,10080.000000,10080.0,10080.000000,10080.000000,10080.000000
mean,30.500000,27.805813,2.440585,81.962500,2.191845,0.612917,0.683333,0.483333,1.950000,11.50000,3.000000,0.285714,8.0,21.134927,79.447959,1.049931
std,17.318961,2.275416,1.785098,11.467641,5.704649,0.867991,0.465199,0.499747,0.825167,6.92253,2.000099,0.451776,0.0,21.159186,2.411122,3.267854
min,1.000000,22.100000,0.000000,44.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,8.0,-28.600000,71.780000,0.000000
25%,15.750000,26.133333,1.200000,74.666667,0.000000,0.000000,0.000000,0.000000,1.000000,5.75000,1.000000,0.000000,8.0,5.300000,77.881755,0.000000
50%,30.500000,27.466667,2.100000,83.000000,0.000000,0.100000,1.000000,0.000000,2.000000,11.50000,3.000000,0.000000,8.0,17.516667,79.214220,0.000000
75%,45.250000,29.100000,3.175000,91.000000,1.500000,1.033333,1.000000,1.000000,3.000000,17.25000,5.000000,1.000000,8.0,33.450000,81.019620,0.000230
max,60.000000,35.400000,22.500000,100.000000,83.500000,3.000000,1.000000,1.000000,3.000000,23.00000,6.000000,1.000000,8.0,88.600000,85.551900,30.285714


# Feature Selection

In [25]:
train_x = train.drop(['date_time'], axis=1)

In [26]:
train_x = train_x.rename({'전력사용량(kWh)':'feature_1', '기온(°C)': 'feature_2', '풍속(m/s)':'feature_3', '습도(%)':'feature_4', '강수량(mm)':'feature_5', '일조(hr)':'feature_6', '비전기냉방설비운영':'feature_7', '태양광보유':'feature_8', 'hour':'feature_9', 'weekday':'feature_10', 'weekend':'feature_11', 'month':'feature_12', 'CDH':'feature_13', 'THI':'feature_14', 'THI_cat':'feature_15', '태양광':'feature_16', 'clst':'feature_17', 'num':'feature_18'}, axis='columns')

In [27]:
train_x

,feature_18,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_17,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,3,0,0,0,6,-8.4,63.43008,1,0.000000
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,0,0,3,1,0,0,6,-16.7,63.56993,1,0.000000
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,0,0,3,2,0,0,6,-25.2,63.22775,1,0.000000
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,0,0,3,3,0,0,6,-34.1,62.54339,1,0.000000
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,0,0,3,4,0,0,6,-43.1,62.39760,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,1,1,2,19,0,0,8,30.8,77.80896,3,2.367647
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,1,1,2,20,0,0,8,32.3,77.44917,3,0.000000
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,1,1,2,21,0,0,8,32.5,77.44917,3,0.000000
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,1,1,2,22,0,0,8,31.3,77.52246,3,0.000000


In [28]:
train_x['feature_15'] = train_x['feature_15'].astype(int)

In [29]:
test_x = test.drop(['date_time'], axis=1)

In [30]:
test_x = test_x.rename({'기온(°C)': 'feature_2', '풍속(m/s)':'feature_3', '습도(%)':'feature_4', '강수량(mm, 6시간)':'feature_5', '일조(hr, 3시간)':'feature_6', '비전기냉방설비운영':'feature_7', '태양광보유':'feature_8', 'hour':'feature_9', 'weekday':'feature_10', 'weekend':'feature_11', 'month':'feature_12', 'CDH':'feature_13', 'THI':'feature_14', 'THI_cat':'feature_15', '태양광':'feature_16', 'clst':'feature_17', 'num':'feature_18'}, axis='columns')

In [31]:
test_x

,feature_18,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_17,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16
0,1,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0,3,0,1,0,8,1.800000,78.602280,3,0.000000
1,1,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0,3,1,1,0,8,3.433333,78.519273,3,0.000000
2,1,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0,3,2,1,0,8,4.900000,78.431867,3,0.000000
3,1,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0,3,3,1,0,8,6.200000,78.340060,3,0.000000
4,1,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0,3,4,1,0,8,7.100000,77.912697,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0,2,19,0,0,8,22.400000,78.764020,3,2.882155
10076,60,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0,2,20,0,0,8,25.400000,78.364280,3,1.525705
10077,60,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,2,21,0,0,8,27.700000,77.957280,3,0.000000
10078,60,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,2,22,0,0,8,29.300000,77.957280,3,0.000000


In [32]:
test_x['feature_15'] = test_x['feature_15'].astype(int)

# Dataset

In [33]:
x_train = train_x.drop(['feature_1'], axis = 1)
y_train = pd.DataFrame(train_x['feature_1'])

print(x_train.shape)
print(y_train.shape)

(122400, 17)
(122400, 1)


In [34]:
test_x['index'] = np.arange(len(test_x))

print(test_x.shape)

(10080, 18)


In [35]:
x_test = test_x.drop(['index'], axis = 1)

print(x_test.shape)

(10080, 17)


# Model

## XGBoost

In [36]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 42)
oof_preds_xgb = np.zeros(train_x.shape[0])

In [37]:
print("xgboost")

sub_preds_xgb = np.zeros(test_x.shape[0])

for train_idx, valid_idx in tqdm(folds.split(x_train, y_train)):
    trn_x = x_train.iloc[train_idx, :]
    trn_y = y_train.iloc[train_idx, :]
    val_x = x_train.iloc[valid_idx, :]
    val_y = y_train.iloc[valid_idx, :]
    
    params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'eta': 0.01, 'max_depth': 10,
              'subsample': 0.6, 'colsample_bytree': 0.6, 'alpha':0.001, 'random_state': 42, 'silent': True}
    
    tr_data = xgb.DMatrix(trn_x, trn_y)
    va_data = xgb.DMatrix(val_x, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 5000, watchlist, maximize=False, early_stopping_rounds = 30, verbose_eval=100)
    
    test_data = xgb.DMatrix(x_test)
    
    oof_preds_xgb[valid_idx] = model_xgb.predict(va_data)
    sub_preds_xgb += model_xgb.predict(test_data) / folds.n_splits
    

submission_xgb['answer'] = sub_preds_xgb
submission_xgb.to_csv('../submission/xgb_{}.csv'.format(version), index = False)

0it [00:00, ?it/s]

xgboost
[23:35:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:36:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:3087.02466	valid-rmse:3084.08960
[100]	train-rmse:1398.12903	valid-rmse:1409.56641
[200]	train-rmse:742.85358	valid-rmse:766.05042
[300]	train-rmse:465.77506	valid-rmse:496.52387
[400]	train-rmse:337.47287	valid-rmse:373.13550
[500]	train-rmse:275.82675	valid-rmse:315.66043
[600]	train-rmse:239.39679	valid-rmse:283.59607
[700]	train-rmse:217.04985	valid-rmse:265.05039
[800]	train-rmse:199.85179	valid-rmse:251.6

1it [04:09, 249.05s/it]

[23:40:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:40:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:3086.34155	valid-rmse:3086.75439
[100]	train-rmse:1400.98791	valid-rmse:1415.72815
[200]	train-rmse:744.66540	valid-rmse:770.68158
[300]	train-rmse:465.95898	valid-rmse:501.18173
[400]	train-rmse:339.02600	valid-rmse:381.99200
[500]	train-rmse:276.16058	valid-rmse:325.58826
[600]	train-rmse:240.04349	valid-rmse:294.64941
[700]	train-rmse:217.68242	valid-rmse:277.15390
[800]	train-rmse:200.70659	valid-rmse:264.80054
[90

2it [08:16, 248.56s/it]

...

3it [12:13, 245.24s/it]

[23:48:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:3084.34619	valid-rmse:3094.82788
[100]	train-rmse:1401.36706	valid-rmse:1423.55920
[200]	train-rmse:747.19812	valid-rmse:780.17883
[300]	train-rmse:468.49033	valid-rmse:506.21494
[400]	train-rmse:341.28986	valid-rmse:380.00299
[500]	train-rmse:279.08728	valid-rmse:319.15476
[600]	train-rmse:242.95281	valid-rmse:284.72937
[700]	train-rmse:220.89696	valid-rmse:265.67227
[800]	train-rmse:204.33491	valid-rmse:252.19684
[90

4it [16:16, 244.42s/it]

[23:52:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:52:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:3079.12329	valid-rmse:3115.06226
[100]	train-rmse:1396.71777	valid-rmse:1422.45850
[200]	train-rmse:743.05756	valid-rmse:771.23016
[300]	train-rmse:466.68543	valid-rmse:501.46115
[400]	train-rmse:338.78091	valid-rmse:381.09042
[500]	train-rmse:277.22784	valid-rmse:326.13828
[600]	train-rmse:240.80243	valid-rmse:295.59738
[700]	train-rmse:217.94026	valid-rmse:278.05316
[800]	train-rmse:200.86760	valid-rmse:265.87042
[90

5it [21:15, 255.09s/it]


## Catboost

In [38]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 42)
oof_preds_catboost = np.zeros(train_x.shape[0])

In [39]:
print("using catboost")

sub_preds_catboost = np.zeros(test_x.shape[0])

for train_idx, valid_idx in tqdm(folds.split(x_train, y_train)):
    trn_x = x_train.iloc[train_idx, :]
    trn_y = y_train.iloc[train_idx, :]
    val_x = x_train.iloc[valid_idx, :]
    val_y = y_train.iloc[valid_idx, :]
    
    cb_model = CatBoostRegressor(iterations=5000, learning_rate=0.05, depth=11, l2_leaf_reg=20,
                                 bootstrap_type='Bernoulli', subsample=0.6, eval_metric='RMSE',
                                 metric_period=50, od_type='Iter', od_wait=20, random_seed=42,
                                 allow_writing_files=False)
    
    cb_model.fit(trn_x, trn_y, eval_set=(val_x, val_y), cat_features=[], use_best_model=True, verbose=True)
    
    oof_preds_catboost[valid_idx] = cb_model.predict(val_x)
    sub_preds_catboost += cb_model.predict(x_test) / folds.n_splits
    

submission_cat['answer'] = sub_preds_catboost
submission_cat.to_csv('../submission/catboost_{}.csv'.format(version), index = False)

0it [00:00, ?it/s]

using catboost


0:	learn: 1991.8882557	test: 1999.1310713	best: 1999.1310713 (0)	total: 216ms	remaining: 18m 1s
50:	learn: 786.5660202	test: 780.4456691	best: 780.4456691 (50)	total: 5.08s	remaining: 8m 12s
100:	learn: 464.9470902	test: 459.6201188	best: 459.6201188 (100)	total: 11.8s	remaining: 9m 33s
150:	learn: 367.2025676	test: 363.3838771	best: 363.3838771 (150)	total: 18.8s	remaining: 10m 2s
200:	learn: 323.0813022	test: 319.9343488	best: 319.9343488 (200)	total: 25.8s	remaining: 10m 15s
250:	learn: 293.9530227	test: 291.8468672	best: 291.8468672 (250)	total: 32.7s	remaining: 10m 19s
300:	learn: 273.5993690	test: 272.9019346	best: 272.9019346 (300)	total: 40s	remaining: 10m 24s
350:	learn: 259.3861676	test: 260.3097272	best: 260.3097272 (350)	total: 47.1s	remaining: 10m 23s
400:	learn: 248.0536983	test: 250.4791009	best: 250.4791009 (400)	total: 54s	remaining: 10m 19s
450:	learn: 239.1391356	test: 243.9664572	best: 243.9664572 (450)	total: 1m 1s	remaining: 10m 17s
500:	learn: 231.4171764	test: 2

4150:	learn: 115.8688664	test: 179.4404473	best: 179.4404473 (4150)	total: 9m 51s	remaining: 2m
4200:	learn: 115.2219247	test: 179.1942269	best: 179.1942269 (4200)	total: 9m 58s	remaining: 1m 53s
4250:	learn: 114.6566052	test: 179.0522926	best: 179.0522926 (4250)	total: 10m 5s	remaining: 1m 46s
4300:	learn: 114.0572784	test: 178.8742179	best: 178.8742179 (4300)	total: 10m 13s	remaining: 1m 39s
4350:	learn: 113.4877962	test: 178.6937603	best: 178.6936291 (4349)	total: 10m 20s	remaining: 1m 32s
4400:	learn: 112.9038526	test: 178.5008846	best: 178.5001210 (4398)	total: 10m 27s	remaining: 1m 25s
4450:	learn: 112.2883880	test: 178.2678383	best: 178.2678383 (4450)	total: 10m 34s	remaining: 1m 18s
4500:	learn: 111.7279388	test: 178.0772136	best: 178.0772136 (4500)	total: 10m 42s	remaining: 1m 11s
4550:	learn: 111.2138729	test: 177.9223066	best: 177.9223066 (4550)	total: 10m 49s	remaining: 1m 4s
4600:	learn: 110.6452001	test: 177.7427368	best: 177.7427368 (4600)	total: 10m 56s	remaining: 56.9s

1it [11:58, 718.17s/it]Warning: Overfitting detector is active, thus evaluation metric is calculated on every iteration. 'metric_period' is ignored for evaluation metric.


...

4150:	learn: 114.8002594	test: 171.7889142	best: 171.7889142 (4150)	total: 9m 53s	remaining: 2m 1s
4200:	learn: 114.2268426	test: 171.6146704	best: 171.6146704 (4200)	total: 10m	remaining: 1m 54s
4250:	learn: 113.6369401	test: 171.4331950	best: 171.4331950 (4250)	total: 10m 8s	remaining: 1m 47s
4300:	learn: 113.0578098	test: 171.2812392	best: 171.2812392 (4300)	total: 10m 15s	remaining: 1m 39s
4350:	learn: 112.4458428	test: 171.1016489	best: 171.1016489 (4350)	total: 10m 22s	remaining: 1m 32s
4400:	learn: 111.7573773	test: 170.8568056	best: 170.8568056 (4400)	total: 10m 29s	remaining: 1m 25s
4450:	learn: 111.0972232	test: 170.6358324	best: 170.6358324 (4450)	total: 10m 37s	remaining: 1m 18s
4500:	learn: 110.4924898	test: 170.4586054	best: 170.4586054 (4500)	total: 10m 44s	remaining: 1m 11s
4550:	learn: 109.9261554	test: 170.3142026	best: 170.3142026 (4550)	total: 10m 51s	remaining: 1m 4s
4600:	learn: 109.2495247	test: 170.0738291	best: 170.0738291 (4600)	total: 10m 58s	remaining: 57.1s

4it [47:52, 718.63s/it]Warning: Overfitting detector is active, thus evaluation metric is calculated on every iteration. 'metric_period' is ignored for evaluation metric.


0:	learn: 1988.3399329	test: 2012.2179081	best: 2012.2179081 (0)	total: 145ms	remaining: 12m 4s
50:	learn: 794.0009399	test: 794.9028952	best: 794.9028952 (50)	total: 4.74s	remaining: 7m 40s
100:	learn: 464.5971884	test: 470.0854584	best: 470.0854584 (100)	total: 11.4s	remaining: 9m 12s
150:	learn: 359.2810714	test: 369.4895419	best: 369.4895419 (150)	total: 18.4s	remaining: 9m 49s
200:	learn: 316.8440620	test: 330.6369771	best: 330.6369771 (200)	total: 25.2s	remaining: 10m 2s
250:	learn: 288.5750404	test: 305.5242224	best: 305.5242224 (250)	total: 32.4s	remaining: 10m 13s
300:	learn: 269.6650441	test: 288.7862388	best: 288.7862388 (300)	total: 39.4s	remaining: 10m 14s
350:	learn: 257.6163536	test: 278.5892474	best: 278.5892474 (350)	total: 46.3s	remaining: 10m 13s
400:	learn: 246.5672812	test: 269.6288008	best: 269.6288008 (400)	total: 53.5s	remaining: 10m 14s
450:	learn: 238.5374039	test: 262.6868909	best: 262.6868909 (450)	total: 1m	remaining: 10m 12s
500:	learn: 231.8229464	test: 2

4150:	learn: 115.3089065	test: 193.2557071	best: 193.2557071 (4150)	total: 9m 50s	remaining: 2m
4200:	learn: 114.7300669	test: 193.0708077	best: 193.0707313 (4199)	total: 9m 57s	remaining: 1m 53s
4250:	learn: 114.0186851	test: 192.8727510	best: 192.8727510 (4250)	total: 10m 4s	remaining: 1m 46s
4300:	learn: 113.4631995	test: 192.7056206	best: 192.7056206 (4300)	total: 10m 12s	remaining: 1m 39s
4350:	learn: 112.9074867	test: 192.4914728	best: 192.4914728 (4350)	total: 10m 19s	remaining: 1m 32s
4400:	learn: 112.3714951	test: 192.2899571	best: 192.2899571 (4400)	total: 10m 26s	remaining: 1m 25s
4450:	learn: 111.8426441	test: 192.1280105	best: 192.1280105 (4450)	total: 10m 33s	remaining: 1m 18s
4500:	learn: 111.1767224	test: 191.8849793	best: 191.8849793 (4500)	total: 10m 40s	remaining: 1m 11s
4550:	learn: 110.5897117	test: 191.6811948	best: 191.6811948 (4550)	total: 10m 48s	remaining: 1m 3s
4600:	learn: 109.9935523	test: 191.5161745	best: 191.5161745 (4600)	total: 10m 55s	remaining: 56.8s

5it [59:48, 717.72s/it]


## LGBM

In [40]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 42)
oof_preds_lgm = np.zeros(train_x.shape[0])

In [41]:
print("lightgbm")

sub_preds_lgm = np.zeros(test_x.shape[0])

for train_idx, valid_idx in tqdm(folds.split(x_train, y_train)):
    trn_x = x_train.iloc[train_idx, :]
    trn_y = y_train.iloc[train_idx, :]
    val_x = x_train.iloc[valid_idx, :]
    val_y = y_train.iloc[valid_idx, :]
    
    params = {'learning_rate': 0.005, 'max_depth': 16, 'boosting': 'dart', 'objective': 'regression',
              'metric': 'rmse', 'is_training_metric': True, 'num_leaves': 144, 'feature_fraction': 0.9,
              'bagging_fraction': 0.7, 'bagging_freq': 6, 'seed':42}
    
    train_T = lgb.Dataset(trn_x, label=trn_y)
    val_T = lgb.Dataset(val_x, label=val_y)
    
    model1 = lgb.train(params, train_T, 5000, val_T, verbose_eval=100, early_stopping_rounds=30)
    
    oof_preds_lgm[valid_idx] = model1.predict(val_x)
    sub_preds_lgm += model1.predict(x_test) / folds.n_splits
    

submission_lgbm['answer'] = sub_preds_lgm
submission_lgbm.to_csv('../submission/lgbm_{}.csv'.format(version), index = False)

0it [00:00, ?it/s]

lightgbm
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006613 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1553
[LightGBM] [Info] Number of data points in the train set: 97920, number of used features: 17
[LightGBM] [Info] Start training from score 2327.154754


D:\Anaconda3\envs\boost\lib\site-packages\lightgbm\callback.py:183: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's rmse: 2122.71
[200]	valid_0's rmse: 2171.96
[300]	valid_0's rmse: 1966.69
[400]	valid_0's rmse: 1866.9
[500]	valid_0's rmse: 1720.83
[600]	valid_0's rmse: 1620.83
[700]	valid_0's rmse: 1415.09
[800]	valid_0's rmse: 1239.16
[900]	valid_0's rmse: 1094.77
[1000]	valid_0's rmse: 1039.72
[1100]	valid_0's rmse: 914.78
[1200]	valid_0's rmse: 902.126
[1300]	valid_0's rmse: 863.107
[1400]	valid_0's rmse: 792.114
[1500]	valid_0's rmse: 770.399
[1600]	valid_0's rmse: 670.548
[1700]	valid_0's rmse: 638.855
[1800]	valid_0's rmse: 638.666
[1900]	valid_0's rmse: 605.353
[2000]	valid_0's rmse: 561.828
[2100]	valid_0's rmse: 555.896
[2200]	valid_0's rmse: 550.916
[2300]	valid_0's rmse: 515.479
[2400]	valid_0's rmse: 511.393
[2500]	valid_0's rmse: 500.674
[2600]	valid_0's rmse: 481.796
[2700]	valid_0's rmse: 457.883
[2800]	valid_0's rmse: 463.827
[2900]	valid_0's rmse: 438.389
[3000]	valid_0's rmse: 414.933
[3100]	valid_0's rmse: 406.068
[3200]	valid_0's rmse: 394.084
[3300]	valid_0's rm

1it [03:36, 216.29s/it]

...

3it [10:40, 213.32s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004909 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1551
[LightGBM] [Info] Number of data points in the train set: 97920, number of used features: 17
[LightGBM] [Info] Start training from score 2324.180464
[100]	valid_0's rmse: 2130.94
[200]	valid_0's rmse: 2180.02
[300]	valid_0's rmse: 1973.82
[400]	valid_0's rmse: 1873.06
[500]	valid_0's rmse: 1726.31
[600]	valid_0's rmse: 1626.35
[700]	valid_0's rmse: 1420.26
[800]	valid_0's rmse: 1244
[900]	valid_0's rmse: 1099.78
[1000]	valid_0's rmse: 1044.07
[1100]	valid_0's rmse: 919.073
[1200]	valid_0's rmse: 906.528
[1300]	valid_0's rmse: 866.714
[1400]	valid_0's rmse: 795.047
[1500]	valid_0's rmse: 772.655
[1600]	valid_0's rmse: 672.103
[1700]	valid_0's rmse: 640.461
[1800]	valid_0's rmse: 640.281
[1900]	valid_0's rmse: 606.764
[2000]	vali

4it [14:09, 212.02s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005765 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1552
[LightGBM] [Info] Number of data points in the train set: 97920, number of used features: 17
[LightGBM] [Info] Start training from score 2319.888826
[100]	valid_0's rmse: 2145.86
[200]	valid_0's rmse: 2193.78
[300]	valid_0's rmse: 1984.86
[400]	valid_0's rmse: 1882.64
[500]	valid_0's rmse: 1733.89
[600]	valid_0's rmse: 1632.95
[700]	valid_0's rmse: 1424.84
[800]	valid_0's rmse: 1247.28
[900]	valid_0's rmse: 1100.65
[1000]	valid_0's rmse: 1044.32
[1100]	valid_0's rmse: 918.418
[1200]	valid_0's rmse: 905.687
[1300]	valid_0's rmse: 866.379
[1400]	valid_0's rmse: 794.619
[1500]	valid_0's rmse: 772.489
[1600]	valid_0's rmse: 671.863
[1700]	valid_0's rmse: 640.451
[1800]	valid_0's rmse: 640.132
[1900]	valid_0's rmse: 606.641
[2000]	v

5it [17:52, 214.41s/it]


## Stacking

### validation prediction

In [42]:
new_data_v = np.array([oof_preds_catboost, oof_preds_lgm, oof_preds_xgb])
new_data_v = np.transpose(new_data_v)
new_data_v.shape

(122400, 3)

In [43]:
regr = linear_model.LinearRegression(fit_intercept = False)
stackedmodel = regr.fit(new_data_v, y_train)

In [44]:
# validation prediction
stackedmodel_pred_v = stackedmodel.predict(new_data_v)
stackedmodel_pred_v.shape

(122400, 1)

In [45]:
def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [46]:
smape(y_train, stackedmodel_pred_v)

feature_1    5.696281
dtype: float64

### test prediction

In [47]:
new_data_t = np.array([sub_preds_catboost, sub_preds_lgm, sub_preds_xgb])
new_data_t = np.transpose(new_data_t)
new_data_t.shape

(10080, 3)

In [48]:
# test prediction
stackedmodel_pred_t = stackedmodel.predict(new_data_t)
stackedmodel_pred_t.shape

(10080, 1)

In [49]:
submission_stack = submission.copy()

In [50]:
submission_stack['answer'] = stackedmodel_pred_t

In [51]:
submission_stack.to_csv('../submission/cat_lgbm_xgb_stack_{}.csv'.format(version), index=False)

In [52]:
submission_stack.describe()

,answer
count,10080.000000
mean,2555.028797
std,2203.750303
min,237.730665
25%,1218.586374
50%,1898.767462
75%,3035.646768
max,17304.385637
